## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-02-11-03-00 +0000,wsj,Gold’s Decline Seen as Healthy Correction Afte...,https://www.wsj.com/finance/commodities-future...
1,2026-02-02-11-00-35 +0000,wapo,Apartment near ICE center sues agency for maki...,https://www.washingtonpost.com/nation/2026/02/...
2,2026-02-02-11-00-28 +0000,wapo,"For traditional Catholics, Charlotte Communion...",https://www.washingtonpost.com/nation/2026/02/...
3,2026-02-02-11-00-00 +0000,nypost,"‘Deadbeat’ hedge fund boss, sued by his own mo...",https://nypost.com/2026/02/02/business/deadbea...
4,2026-02-02-11-00-00 +0000,wsj,Why Prices at Five Below Are Nudging Above $5,https://www.wsj.com/articles/why-prices-at-fiv...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2584/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
129,trump,27
9,ice,20
170,epstein,19
110,new,13
222,year,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
106,2026-02-02-02-56-44 +0000,nyt,Trump Says Kennedy Center Will Close for 2-Yea...,https://www.nytimes.com/2026/02/01/us/politics...,67
144,2026-02-01-22-34-48 +0000,nypost,India to buy oil from Venezuela rather than Ru...,https://nypost.com/2026/02/01/world-news/india...,65
211,2026-02-01-15-09-48 +0000,nypost,Trump jokes about suing new Fed chair nominee ...,https://nypost.com/2026/02/01/us-news/trump-jo...,64
108,2026-02-02-02-49-20 +0000,nypost,"5-year-old boy, father taken into custody by I...",https://nypost.com/2026/02/01/us-news/5-year-o...,62
121,2026-02-02-01-24-00 +0000,wsj,President Trump says the Kennedy Center will b...,https://www.wsj.com/politics/elections/trump-s...,62


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
106,67,2026-02-02-02-56-44 +0000,nyt,Trump Says Kennedy Center Will Close for 2-Yea...,https://www.nytimes.com/2026/02/01/us/politics...
97,51,2026-02-02-03-38-34 +0000,startribune,The latest: Musicians call out ICE actions in ...,https://www.startribune.com/ice-raids-minnesot...
74,49,2026-02-02-06-29-11 +0000,nypost,Former UK minister Mandelson quits Labour afte...,https://nypost.com/2026/02/02/world-news/uk-mi...
8,36,2026-02-02-10-51-21 +0000,bbc,Israel reopens Gaza's key Rafah border crossin...,https://www.bbc.com/news/articles/c99kdjdj9l8o...
16,35,2026-02-02-10-30-00 +0000,wsj,"Heard on the Street: Kevin Warsh, President Tr...",https://www.wsj.com/economy/central-banking/sa...
108,30,2026-02-02-02-49-20 +0000,nypost,"5-year-old boy, father taken into custody by I...",https://nypost.com/2026/02/01/us-news/5-year-o...
217,30,2026-02-01-14-26-13 +0000,cbc,More than a dozen people killed in Ukraine aft...,https://www.cbc.ca/news/world/ukraine-talks-ru...
124,28,2026-02-02-01-12-32 +0000,nypost,Ex-Gov. Chris Christie claims Trump is politic...,https://nypost.com/2026/02/01/us-news/trump-is...
92,27,2026-02-02-04-43-48 +0000,nypost,Paul Weiss law firm chair Brad Karp told Jeffr...,https://nypost.com/2026/02/01/us-news/paul-wei...
56,26,2026-02-02-08-40-00 +0000,wsj,Oil Prices Tumble on Broader Commodities Sello...,https://www.wsj.com/finance/commodities-future...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
